## ALMA inference

The purpose of this project is to use ALMA model (write simple program to inference it through localhost web ui - no langchain, no complicated libraries, just simple webapp). 

This project could be also a way to test some solutions for web interfaces of more complicated DocWhisper project.

1) we quantize model to smaller size (if X-ALMA will not be possible we could use GGUF version) - in other project we will learn how to quantize models to for example GGUF ourselves (patrz. BIBLIOTEKA_transformers-GPTs)
    - how to quatize model?
    - how to save it to for example GGUF?
2) we distill knowledge to smaller model
3) we create simple algo for inference
4) we import our custom model 

It is not worth to lower precision to int8 for pre-quantize model for now (I might look into it in a future)

In [ ]:
# Text to przetłumaczenia
# Tak więc MoE łączy kilka innych modeli, aby wybrać, który ślad (część modelu) powinna zostać użyta do uzyskania prawidłowej odpowiedzi. Jest to technika optymalizacji. Agentic Workflow z drugiej strony to wiele niezależnych modeli, które działają w ramach jakiejś aplikacji, np. jeśli dane wejściowe to obraz, działają agenci wizji komputerowej itp. Bagging trenuje kilka różnych modeli, każdy na innym zbiorze danych i uśrednia wyniki, a boosting używa kilku różnych modeli, z których każdy poprawia błędy poprzedniego. Czy to wyjaśnienie jest poprawne?

In [1]:
import torch
import pathlib
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from transformers import pipeline
from llama_cpp import Llama
import textract

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
cache_dir = pathlib.Path.cwd() / "Model"

GROUP2LANG = {
1: ["da", "nl", "de", "is", "no", "sv", "af"],
2: ["ca", "ro", "gl", "it", "pt", "es"],
3: ["bg", "mk", "sr", "uk", "ru"],
4: ["id", "ms", "th", "vi", "mg", "fr"],
5: ["hu", "el", "cs", "pl", "lt", "lv"],
6: ["ka", "zh", "ja", "ko", "fi", "et"],
7: ["gu", "hi", "mr", "ne", "ur"],
8: ["az", "kk", "ky", "tr", "uz", "ar", "he", "fa"],
}

LANG2GROUP = {lang: group for group, langs in GROUP2LANG.items() for lang in langs}
group_id = LANG2GROUP["pl"]

model_name = f"haoranxu/X-ALMA-13B-Group{group_id}"

In [4]:
!accelerate estimate-memory haoranxu/X-ALMA-13B-Group5 --library_name transformers

Loading pretrained config for `haoranxu/X-ALMA-13B-Group5` from `transformers`...
┌──────────────────────────────────────────────────────┐
│Memory Usage for loading `haoranxu/X-ALMA-13B-Group5` │
├───────┬─────────────┬──────────┬─────────────────────┤
│ dtype │Largest Layer│Total Size│ Training using Adam │
├───────┼─────────────┼──────────┼─────────────────────┤
│float32│   1.18 GB   │ 47.88 GB │      191.51 GB      │
│float16│  605.02 MB  │ 23.94 GB │       95.76 GB      │
│  int8 │  302.51 MB  │ 11.97 GB │         N/A         │
│  int4 │  151.25 MB  │ 5.98 GB  │         N/A         │
└───────┴─────────────┴──────────┴─────────────────────┘


In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
# II) way of loading model
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, cache_dir=cache_dir).to(device)  
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", cache_dir=cache_dir)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
prompt="Translate this from Polish to English:\nPolish: Tak więc MoE łączy kilka innych modeli, aby wybrać, który ślad (część modelu) powinna zostać użyta do uzyskania prawidłowej odpowiedzi. Czy to wyjaśnienie jest poprawne? \nEnglish"

chat_style_prompt = [{"role": "user", "content": prompt}]
prompt = tokenizer.apply_chat_template(chat_style_prompt, tokenize=False, add_generation_prompt=True)

input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=600, truncation=True).input_ids.to(device)

In [11]:
# Actual translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=200, do_sample=True, temperature=0.6, top_p=0.9)
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    print(outputs)

['[INST] Translate this from Polish to English:\nPolish: Tak więc MoE łączy kilka innych modeli, aby wybrać, który ślad (część modelu) powinna zostać użyta do uzyskania prawidłowej odpowiedzi. Czy to wyjaśnienie jest poprawne? \nEnglish [/INST]Thus, the MoE combines several other models to choose which trace (part of the model) should be used to obtain the correct answer. Is this explanation correct?']


### Loading entire file

In [7]:
def extract_text_from_file(file_path):
    return textract.process(file_path).decode("utf-8")

file_path = pathlib.Path.cwd() / "Test_docs/Praca_magisterska.docx"
text = extract_text_from_file(file_path)

In [8]:
chat_style_prompt = [{"role": "user", "content": text}]
prompt = tokenizer.apply_chat_template(chat_style_prompt, tokenize=False, add_generation_prompt=True)

input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=600, truncation=True).input_ids.to(device)

In [9]:
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=200, do_sample=True, temperature=0.6, top_p=0.9)
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

output_file = "output.txt"
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(outputs))

In [10]:
# TODO:
# 1) wstawić automatyczne dostosowywanie max_new_tokens i max_lenght tak aby model zwracał output dla całego tekstu a nie tylko pierwszych 600 znakow
#    - również funkcja powinna wczytywać dane tak aby oszczędzać miejsce w pamięci (coś w rodzaju batchy)
# 2) przygotować prosty interfejs webowy w gradio